In [1]:
import csv          # Python package for reading and writing CSV files.
import copy         # Copy data structures.
import json

import sys,os

# You can change to wherever you want to place your CSV files.
rel_path = os.path.realpath('./data')
rel_path += '/'

In [3]:
print(type(rel_path))

<class 'str'>


In [19]:
class CSVTable():

    # Change to wherever you want to save the CSV files.

    def __init__(self, table_name, table_file, key_columns):
        '''
        Constructor
        :param table_name: Logical names for the data table.
        :param table_file: File name of CSV file to read/write.
        :param key_columns: List of column names the form the primary key.
        '''
        self.table_name = table_name
        self.table_file = rel_path + table_file
        self.key_columns = key_columns
        self.data = []
    
    def __str__(self):
        '''
        Pretty print the table and state.
        :return: String
        '''
        return json.dumps(self.data, indent=2)
        
    def load(self):
        '''
        Load information from CSV file.
        :return: None
        '''
        try:
            with open(self.table_file, mode = 'r') as csv_file:
                csv_reader = csv.DictReader(csv_file)
                for i in csv_reader:
                    self.data.append(i)
        except IOError:
            print ("The datatable file doens't exit")
        
        for i in self.key_columns:
            if i not in self.data[0]:
                raise ValueError("The primary keys are invalid")

    def find_by_primary_key(self, string_set, fields=None):
        
        if len(string_set) != len(self.key_columns):
            raise ValueError("The primary keys inputed didn't match the key columns")

        if fields == None:
            for i in self.data:
                count_match = 0
                for j in range(0,len(string_set)):
                    if i[self.key_columns[j]] == string_set[j]:
                        count_match += 1
                if count_match == len(self.key_columns):
                    return i
        else:
            
            for j in fields:
                if j not in self.data[0].keys():
                    raise ValueError("The fields don't match the database column types")
            
            unwanted = self.data[0].keys() - fields
            for i in self.data:
                count_match = 0
                for j in range(0,len(string_set)):
                    if i[self.key_columns[j]] == string_set[j]:
                        count_match += 1
                if count_match == len(self.key_columns):
                    for unwanted_key in unwanted:
                        del i[unwanted_key]
                    return i

        return None        

    def find_by_template(self, t, fields=None):
        '''
        Return a table containing the rows matching the template and field selector.
        :param t: Template that the rows much match.
        :param fields: A list of columns to include in responses.
        :return: CSVTable containing the answer.
        '''
        t_keys = t.keys()
        for i in t_keys:
            if i not in self.data[0]:
                raise ValueError("Keys in templated doesn't match the database format")


        result = []
        if fields == None:
            keys = t.keys()
            for i in self.data:
                count_match = 0
                for key in keys:
                    if i[key] == t[key]:
                        count_match += 1
                
                if count_match == len(keys):
                    result.append(i)
        else:
            
            for j in fields:
                if j not in self.data[0].keys():
                    raise ValueError("The fields don't match the database column types")
            
            keys = t.keys()
            unwanted = self.data[0].keys() - fields
            for i in self.data:
                count_match = 0
                for key in keys:
                    if i[key] == t[key]:
                        count_match += 1
                
                if count_match == len(keys):
                    for unwanted_key in unwanted:
                        del i[unwanted_key]
                    result.append(i)
        
        return result

    def find_by_template_top(self):
        result = set()

        for i in self.data:
            if int(i["yearID"]) >= 1960 and int(i["AB"]) >= 200:
                result.add(i['playerID'])
        
        return result

    def save(self):
        '''
        Write updated CSV back to the original file location.
        :return: None
        '''
        try:
            with open(self.table_file, mode='w', newline = '') as csv_file:
                fieldnames = self.data[0].keys()
                writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
                writer.writeheader()
                for i in self.data:
                    writer.writerow(i)
        except IOError:
            print( "Can't open the file to write" )

    def insert(self, r):
        '''
        Insert a new row into the table.
        :param r: New row.
        :return: None. Table state is updated.
        '''
        t_keys = r.keys()
        for i in t_keys:
            if i not in self.data[0]:
                raise ValueError("Keys in templated doesn't match the database format")
        
        primary_key_string_set = []

        for i in self.key_columns:
            primary_key_string_set.append(r[i])

        result = self.find_by_primary_key(primary_key_string_set)

        if result != None:
            raise ValueError("Duplicated Primary key found, Not insert!")

        self.data.append(r)


    def delete(self, t):
        '''
        Delete all tuples matching the template.
        :param t: Template
        :return: None. Table is updated.
        '''
        t_keys = t.keys()
        for i in t_keys:
            if i not in self.data[0]:
                raise ValueError("Keys in templated doesn't match the database format")
        
        keys = t.keys()
        for i in self.data:
            count_match = 0
            for key in keys:
                if i[key] == t[key]:
                    count_match += 1
            
            if count_match == len(keys):
                self.data.remove(i)

In [20]:
def test1():

    csvt = CSVTable("People", "PeopleSmall.csv", ["playerID"])
    csvt.load()
    print("Table = ", csvt)


def test_template(test_name, table_name, table_file, key_columns, template, fields=None, show_rows=False):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template = ", template)
    print("Fields = ", fields)

    try:
        csvt = CSVTable(table_name, table_file, key_columns)
        csvt.load()

        if not show_rows:
            print("Table name = ", csvt.table_name)
            print("Table file = ", csvt.table_file)
            print("Table keys = ", csvt.key_columns)
        else:
            print(csvt)

        r = csvt.find_by_template(template, fields)
        print("Result table:")
        print(json.dumps(r, indent=2))
        
    except ValueError as ve:
        print("Exception = ", ve)

def test_delete( test_name, table_name, table_file, key_columns, template ):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template to delete = ", template)

    try:
        csvt = CSVTable(table_name, table_file, key_columns)
        csvt.load()
        csvt.delete(template)
        csvt.save()
    except ValueError as ve:
        print("Exception = ", ve)

def test_insert(test_name, table_name, table_file, key_columns, row, show_rows=False):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)

    try:
        csvt = CSVTable(table_name, table_file, key_columns)
        csvt.load()

        if not show_rows:
            print("Table name = ", csvt.table_name)
            print("Table file = ", csvt.table_file)
            print("Table keys = ", csvt.key_columns)
        else:
            print(csvt)

        r = csvt.insert(row)
        print("Result table:")

        csvt.save()

        if show_rows:
            print(json.dumps(r, indent=2))

    except ValueError as ve:
        print("Exception = ", ve)

def test_primary_key(test_name, table_name, table_file, key_columns,string_set,fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Primary key = ", string_set)
    try:
        csvt = CSVTable(table_name, table_file, key_columns)
        csvt.load()
        r = csvt.find_by_primary_key(string_set,fields)
        print("Result table:")
        print(json.dumps(r, indent=2))

    except ValueError as ve:
        print("Exception = ", ve)
        

def test_templates():
    test_template("Test2", "People", "People.csv", ["playerID"],
                  {"birthMonth": "9", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    test_template("Test3", "People", "People.csv", ["playerID"],
                  {"nameFirst": "Ted", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    test_template("Test4", "People", "People.csv", ["canary"],
                  {"nameFirst": "Ted", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    test_template("Test5", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"],
                  False)

    test_template("Test6", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "iq": 100}, ["playerID", "yearID", "teamID", "AB", "H", "HR"],
                  False)

    test_template("Test7", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "yearID": "1961"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"],
                  False)

    test_template("Test7", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "yearID": "1960"}, ["playerID", "yearID", "teamID", "AB", "H", "HR", "Age"],
                  False)


def test_inserts():

    test_insert("Insert Test 1", "People", "PeopleSmall.csv", ["playerID"],
                {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
                False)

    test_template("Find after insert 1", "People", "PeopleSmall.csv", ["playerID"],
                  {"nameLast": "Ferguson"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    try:
        test_insert("Insert Test 2", "People", "PeopleSmall.csv", ["playerID"],
                    {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
                    False)

        raise ValueError("That insert should not have worked!")

    except ValueError as ve:
        print("OK. Did not insert duplicate key.")


    test_insert("Insert Test 3", "Batting", "BattingSmall.csv", ["playerID", "yearID", "teamID", "stint"],
                {"playerID": "dff1", "teamID": "BOS", "yearID": "2018", "stint": "1",
                    "AB": "100", "H": "100"},
                False)

    test_template("Find after insert 3", "Batting", "BattingSmall.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "dff1"}, None,
                  False)


def test_deletes():
    test_delete("Delete Test 1", "People", "PeopleSmall.csv", ["playerID"],
                {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"})
    try:
        test_delete("Delete Test 1", "People", "PeopleSmall.csv", ["playerID"],
                {"playerID": "dff2", "nameLast": "Ferguson", "nameFirst": "Donald"})
        raise ValueError("That insert should not have worked!")
    
    except ValueError as ve:
        print("OK. Did not delete the data which in not in the CSV FILE.")

def test_find_by_primary_key():
    test_primary_key("Primary Key Test1", "People", "PeopleSmall.csv", ["playerID"],
                      ["dff_chi"])
    
    test_primary_key("Primary Key Test2", "People", "PeopleSmall.csv", ["playerID"],
                      ["abbotji01"])

In [21]:
test_find_by_primary_key()



*******************************
Test name =  Primary Key Test1
Primary key =  ['dff_chi']
Result table:
null


*******************************
Test name =  Primary Key Test2
Primary key =  ['abbotji01']
Result table:
{
  "playerID": "abbotji01",
  "birthYear": "1967",
  "birthMonth": "9",
  "birthDay": "19",
  "birthCountry": "USA",
  "birthState": "MI",
  "birthCity": "Flint",
  "deathYear": "",
  "deathMonth": "",
  "deathDay": "",
  "deathCountry": "",
  "deathState": "",
  "deathCity": "",
  "nameFirst": "Jim",
  "nameLast": "Abbott",
  "nameGiven": "James Anthony",
  "weight": "200",
  "height": "75",
  "bats": "L",
  "throws": "L",
  "debut": "4/8/1989",
  "finalGame": "7/21/1999",
  "retroID": "abboj001",
  "bbrefID": "abbotji01"
}


In [22]:
test_templates()



*******************************
Test name =  Test2
Template =  {'birthMonth': '9', 'nameLast': 'Williams'}
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Table name =  People
Table file =  D:\Courses\Introduction to Database\Projects\HW1\data/People.csv
Table keys =  ['playerID']
Result table:
[
  {
    "birthYear": "1968",
    "birthMonth": "9",
    "nameFirst": "Bernie",
    "nameLast": "Williams"
  },
  {
    "birthYear": "1931",
    "birthMonth": "9",
    "nameFirst": "Don",
    "nameLast": "Williams"
  },
  {
    "birthYear": "1935",
    "birthMonth": "9",
    "nameFirst": "Don",
    "nameLast": "Williams"
  },
  {
    "birthYear": "1993",
    "birthMonth": "9",
    "nameFirst": "Nick",
    "nameLast": "Williams"
  },
  {
    "birthYear": "1975",
    "birthMonth": "9",
    "nameFirst": "Randy",
    "nameLast": "Williams"
  },
  {
    "birthYear": "1936",
    "birthMonth": "9",
    "nameFirst": "Stan",
    "nameLast": "Williams"
  }
]


***************************

In [23]:
test_inserts()



*******************************
Test name =  Insert Test 1
Row to insert =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
Table name =  People
Table file =  D:\Courses\Introduction to Database\Projects\HW1\data/PeopleSmall.csv
Table keys =  ['playerID']
Result table:


*******************************
Test name =  Find after insert 1
Template =  {'nameLast': 'Ferguson'}
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Table name =  People
Table file =  D:\Courses\Introduction to Database\Projects\HW1\data/PeopleSmall.csv
Table keys =  ['playerID']
Result table:
[
  {
    "birthYear": "",
    "birthMonth": "",
    "nameFirst": "Donald",
    "nameLast": "Ferguson"
  }
]


*******************************
Test name =  Insert Test 2
Row to insert =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
Table name =  People
Table file =  D:\Courses\Introduction to Database\Projects\HW1\data/PeopleSmall.csv
Table keys =  ['playerID']
Exceptio

In [24]:
test_deletes()



*******************************
Test name =  Delete Test 1
Template to delete =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}


*******************************
Test name =  Delete Test 1
Template to delete =  {'playerID': 'dff2', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
OK. Did not delete the data which in not in the CSV FILE.


In [10]:
import pymysql
import json

In [11]:
def templateToWhereClause(t):
    s = ""
    for k,v in t.items():
        if s != "":
            s += " AND "
        s += k + "='" + v + "'"

    if s != "":
        s = "WHERE " + s

    return s
def templateToInsertClause(t):
    s = "("
    keys = t.keys()
    count = 0
    for i in keys:
        s += i
        count += 1
        if count != len(keys):
            s += ","

    s += ") VALUES ("
    count = 0
    for i in keys:
        s += "'" + t[i] + "'"
        count += 1
        if count != len(keys): 
            s += ","
    s+= ")"

    return s

In [12]:
class RDBDataTable():
    def __init__(self, host, user, password, db_name, table_name, key_columns ):
        # Your code goes here
        self.cnx = pymysql.connect(host= host,
                              user= user,
                              password= password,
                              db=db_name,
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)
        self.cursor = self.cnx.cursor()
        self.table_name = table_name
        self.key_columns = key_columns
        self.columns = []
        
        q = "SHOW columns FROM " + table_name + ";"
        self.cursor.execute(q)
        r = self.cursor.fetchall()
        for i in r:
            self.columns.append(i["Field"])

        for i in key_columns:
            if i not in self.columns:
                raise ValueError("The key_columns dones't math the database columns type")

    def find_by_primary_key(self, string_set, fields=None):
        if len(string_set) != len(self.key_columns):
            raise ValueError("The primary keys inputed didn't match the key columns")
        
        w = ""
        for i in range(0,len(string_set)):
            if w != "":
                w += " AND "
            w += self.key_columns[i] + "='" + string_set[i] + "'"
        if w != "":
            w = "WHERE " + w

        if fields == None:
            q = "SELECT * FROM " + self.table_name + " " + w + ";"
        else:
            for j in fields:
                if j not in self.columns:
                    raise ValueError("The fields don't match the database column types")
            
            fields_string = ""
            for i in fields:
                fields_string += i + ","
            fields_string = fields_string[:-1]

            q = "SELECT " + fields_string + " FROM " + self.table_name + " " + w + ";"
        
        # print ("Query = ", q)
        self.cursor.execute(q)
        r = self.cursor.fetchall()
        return r

    # The input is:
    # t: The template to match. The result is a list of rows
    # whose attribute/value pairs exactly match the template.
    # fields: A subset of the fields to include for each result.
    # Raises an exception if the template or list of fields contains
    # a column/attribute name not in the file.
    def find_by_template(self, t, fields=None):

        t_keys = t.keys()
        for i in t_keys:
            if i not in self.columns:
                raise ValueError("Keys in templated doesn't match the database column types")

        
        
        w = templateToWhereClause(t)

        if fields == None:
            q = "SELECT * FROM " + self.table_name + " " + w + ";"
        else:
            for j in fields:
                if j not in self.columns:
                    raise ValueError("The fields don't match the database column types")
            
            fields_string = ""
            for i in fields:
                fields_string += i + ","
            fields_string = fields_string[:-1]

            q = "SELECT " + fields_string + " FROM " + self.table_name + " " + w + ";"

        # print ("Query = ", q)
        self.cursor.execute(q)
        r = self.cursor.fetchall()
        return r


    # Inserts the row into the table. 
    # Raises on duplicate key or invalid columns.
    def insert(self, t):
        t_keys = t.keys()
        for i in t_keys:
            if i not in self.columns:
                raise ValueError("Keys in templated doesn't match the database format")
        
        primary_key_string_set = []
        for i in self.key_columns:
            primary_key_string_set.append(t[i])
        
        result = self.find_by_primary_key(primary_key_string_set)
        

        if len(result) != 0:
             raise ValueError("Duplicated Primary key found, Not insert!")

        w = templateToInsertClause(t)
        q = "INSERT INTO " + self.table_name + " " + w + ";"
#         print ("Query = ", q)
        self.cursor.execute(q)
        self.cnx.commit()
       
    # t: A template.
    # Deletes all rows matching the template.
    def delete(self, t):

        t_keys = t.keys()
        for i in t_keys:
            if i not in self.columns:
                raise ValueError("Keys in templated doesn't match the database format")

        w = templateToWhereClause(t)
        q = "DELETE FROM " + self.table_name + " " + w + ";"
        # print ("Query = ", q)
        self.cursor.execute(q)


In [4]:
def test1():
    try:
        RDBtable = RDBDataTable("localhost","root","zhangchi25","lahman2017n",
                                "People", ["playerID"])
    except:
        print("Exception,the connect failed")

In [5]:
RDBtable = RDBDataTable("localhost","root","zhangchi25","lahman2017n",
                                "peoplesmall", ["playerID"])

In [6]:
RDBtable.delete({"playerID": "dff2", "nameLast": "Ferguson", "nameFirst": "Donald"})

In [13]:
def test_template(test_name, table_name, key_columns, template, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template = ", template)
    print("Fields = ", fields)

    try:
        RDBtable = RDBDataTable("localhost","root","zhangchi25","lahman2017n",
                                table_name, key_columns)

        r = RDBtable.find_by_template(template, fields)
        print("Result table:")
        print(json.dumps(r, indent=2))
        
    except ValueError as ve:
        print("Exception = ", ve)

def test_delete( test_name, table_name, key_columns, template ):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template to delete = ", template)

    try:
        RDBtable = RDBDataTable("localhost","root","zhangchi25","lahman2017n",
                                table_name, key_columns)
        RDBtable.delete(template)

    except ValueError as ve:
        print("Exception = ", ve)

def test_insert(test_name, table_name, key_columns, row, show_rows=False):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)

    try:
        RDBtable = RDBDataTable("localhost","root","zhangchi25","lahman2017n",
                                table_name, key_columns)

        r = RDBtable.insert(row)
        print("Result table:")
        print(json.dumps(r, indent=2))

    except ValueError as ve:
        print("Exception = ", ve)

def test_primary_key(test_name, table_name, key_columns,string_set,fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Primary key = ", string_set)
    try:
        RDBtable = RDBDataTable("localhost","root","zhangchi25","lahman2017n",
                                table_name, key_columns)
        r = RDBtable.find_by_primary_key(string_set,fields)
        print("Result table:")
        print(json.dumps(r, indent=2))

    except ValueError as ve:
        print("Exception = ", ve)
        

def test_templates():
    test_template("Test2", "People", ["playerID"],
                  {"birthMonth": "9", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"])

    test_template("Test3", "People", ["playerID"],
                  {"nameFirst": "Ted", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"])

    test_template("Test4", "People", ["canary"],
                  {"nameFirst": "Ted", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"])

    test_template("Test5", "Batting", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"])

    test_template("Test6", "Batting", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "iq": 100}, ["playerID", "yearID", "teamID", "AB", "H", "HR"])

    test_template("Test7", "Batting",  ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "yearID": "1961"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"])

    test_template("Test7", "Batting", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "yearID": "1960"}, ["playerID", "yearID", "teamID", "AB", "H", "HR", "Age"])


def test_inserts():

    test_insert("Insert Test 1", "peoplesmall", ["playerID"],
                {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"})

    test_template("Find after insert 1", "peoplesmall", ["playerID"],
                  {"nameLast": "Ferguson"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"])

    try:
        test_insert("Insert Test 2", "peoplesmall", ["playerID"],
                    {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"})

        raise ValueError("That insert should not have worked!")

    except ValueError as ve:
        print("OK. Did not insert duplicate key.")


    test_insert("Insert Test 3", "battingsmall", ["playerID", "yearID", "teamID", "stint"],
                {"playerID": "dff1", "teamID": "BOS", "yearID": "2018", "stint": "1",
                    "AB": "100", "H": "100"})

    test_template("Find after insert 3", "battingsmall",  ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "dff1"}, None)


def test_deletes():
    test_delete("Delete Test 1", "peoplesmall", ["playerID"],
                {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"})
    try:
        test_delete("Delete Test 1", "peoplesmall", ["playerID"],
                {"playerID": "dff2", "nameLast": "Ferguson", "nameFirst": "Donald"})
        raise ValueError("That delete should not have worked!")
    
    except ValueError as ve:
        print("OK. Did not delete the data which in not in the CSV FILE.")

def test_find_by_primary_key():
    test_primary_key("Primary Key Test1", "People", ["playerID"],
                      ["dff_chi"])
    
    test_primary_key("Primary Key Test2", "People", ["playerID"],
                      ["abbotji01"])

In [13]:
test_templates()



*******************************
Test name =  Test2
Template =  {'birthMonth': '9', 'nameLast': 'Williams'}
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Result table:
[
  {
    "nameLast": "Williams",
    "nameFirst": "Bernie",
    "birthMonth": "9",
    "birthYear": "1968"
  },
  {
    "nameLast": "Williams",
    "nameFirst": "Don",
    "birthMonth": "9",
    "birthYear": "1931"
  },
  {
    "nameLast": "Williams",
    "nameFirst": "Don",
    "birthMonth": "9",
    "birthYear": "1935"
  },
  {
    "nameLast": "Williams",
    "nameFirst": "Nick",
    "birthMonth": "9",
    "birthYear": "1993"
  },
  {
    "nameLast": "Williams",
    "nameFirst": "Randy",
    "birthMonth": "9",
    "birthYear": "1975"
  },
  {
    "nameLast": "Williams",
    "nameFirst": "Stan",
    "birthMonth": "9",
    "birthYear": "1936"
  }
]


*******************************
Test name =  Test3
Template =  {'nameFirst': 'Ted', 'nameLast': 'Williams'}
Fields =  ['nameLast', 'nameFirst', 'birthMont

In [17]:
test_find_by_primary_key()



*******************************
Test name =  Primary Key Test1
Primary key =  ['dff_chi']
Result table:
[]


*******************************
Test name =  Primary Key Test2
Primary key =  ['abbotji01']
Result table:
[
  {
    "playerID": "abbotji01",
    "birthYear": "1967",
    "birthMonth": "9",
    "birthDay": "19",
    "birthCountry": "USA",
    "birthState": "MI",
    "birthCity": "Flint",
    "deathYear": "",
    "deathMonth": "",
    "deathDay": "",
    "deathCountry": "",
    "deathState": "",
    "deathCity": "",
    "nameFirst": "Jim",
    "nameLast": "Abbott",
    "nameGiven": "James Anthony",
    "weight": "200",
    "height": 75,
    "bats": "L",
    "throws": "L",
    "debut": "1989-04-08",
    "finalGame": "1999-07-21",
    "retroID": "abboj001",
    "bbrefID": "abbotji01"
  }
]


In [14]:
test_inserts()



*******************************
Test name =  Insert Test 1
Row to insert =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
Result table:
null


*******************************
Test name =  Find after insert 1
Template =  {'nameLast': 'Ferguson'}
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Result table:
[
  {
    "nameLast": "Ferguson",
    "nameFirst": "Donald",
    "birthMonth": null,
    "birthYear": null
  },
  {
    "nameLast": "Ferguson",
    "nameFirst": "Donald",
    "birthMonth": null,
    "birthYear": null
  }
]


*******************************
Test name =  Insert Test 2
Row to insert =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
Exception =  Duplicated Primary key found, Not insert!
OK. Did not insert duplicate key.


*******************************
Test name =  Insert Test 3
Row to insert =  {'playerID': 'dff1', 'teamID': 'BOS', 'yearID': '2018', 'stint': '1', 'AB': '100', 'H': '100'}
Result table:
null


**

In [15]:
test_deletes()



*******************************
Test name =  Delete Test 1
Template to delete =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}


*******************************
Test name =  Delete Test 1
Template to delete =  {'playerID': 'dff2', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
OK. Did not delete the data which in not in the CSV FILE.
